In [5]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.utils import to_categorical
import numpy as np
import string

## 1. Configuration

In [6]:
IMG_HEIGHT = 64
IMG_WIDTH = 256
BATCH_SIZE = 32
EPOCHS = 50

CHARS = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ -"
CHAR_TO_NUM = {char: i for i, char in enumerate(CHARS)}
NUM_TO_CHAR = {i: char for i, char in enumerate(CHARS)}

## 2. Data Loader

#### 2.1 Load and preprocess image

In [7]:
def preprocess_image(image_path, img_height=IMG_HEIGHT, img_width=IMG_WIDTH):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=1)
    img = tf.image.resize(img, [img_height, img_width])
    img = tf.cast(img, tf.float32) / 255.0
    return img

#### 2.2 Encode text to integer sequence

In [8]:
def encode_text(text):
    return [CHAR_TO_NUM[c] for c in text if c in CHAR_TO_NUM]

#### 2.3 Create tf.data.Dataset from CSV and images

In [9]:
def create_dataset(csv_path, image_dir, img_height=IMG_HEIGHT, img_width=IMG_WIDTH):
    df = pd.read_csv(csv_path)
    filenames = df['filename'].values
    texts = df['plate_number'].values

    def generator():
        for filename, text in zip(filenames, texts):
            img_path = os.path.join(image_dir, filename)
            img = preprocess_image(img_path, img_height, img_width)
            label = encode_text(text)
            yield img, label

    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(img_height, img_width, 1), dtype=tf.float32),
            tf.RaggedTensorSpec(shape=[None], dtype=tf.int32)
        )
    )

    max_len = max(len(t) for t in texts)
    dataset = dataset.map(lambda x, y: (
        x,
        tf.pad(y, [[0, max_len - tf.shape(y)[0]]], constant_values=-1)
    ))

    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset

## 3. MODEL ARCHITECTURE

In [10]:
def create_crnn_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 1), num_chars=len(CHARS)):
    inputs = layers.Input(shape=input_shape, name='image')

    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)

    new_shape = ((input_shape[0] // 8), (input_shape[1] // 8) * 128)
    x = layers.Reshape(target_shape=new_shape)(x)

    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)

    outputs = layers.Dense(num_chars + 1, activation='softmax', name='dense')(
        x)

    model = Model(inputs=inputs, outputs=outputs)
    return model